## K-Means Clustering
The goal is to:
* use the contents of the existing df_rescaled_pitch_info.csv contained standardized information on each type of         pitch that they throw
* use all 217 features (we'll have to make that alterable to build a decent model?)
* and build a file that puts each pitcher in one cluster of "similar pitchers"

## Import all the needed libraries.

In [164]:
!pip install mxnet

import pandas as pd
import numpy as np
from datetime import datetime

import boto3
from sagemaker import get_execution_role
import sagemaker.amazon.common as smac
import mxnet as mx

You are using pip version 10.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [165]:
%%html
<style>
.rendered_html tr, .rendered_html th, .rendered_html td {
  text-align: left;
}
# .rendered_html :first-child {
#   text-align: left;
# }
# .rendered_html :last-child {
#   text-align: left;
# }
</style>

## Loading the data from Amazon S3
Next, lets get the UFO sightings data that is stored in S3.

In [166]:
bucket = 'appleforge-merlin-develop-datalake'
prefix = 'pitchtype'
data_key = 'df_rescaled_pitch_info.csv'
data_location = 's3://{}/{}/{}'.format(bucket, prefix, data_key)

df = pd.read_csv(data_location, low_memory=False)

In [167]:
df.head()

,pitcher,p_throws,2f_ax,2f_ay,2f_az,2f_pct_usage,2f_pfx_x,2f_pfx_z,2f_plate_x,2f_plate_z,...,sl_release_extension,sl_release_pos_x,sl_release_pos_z,sl_release_speed,sl_release_spin_rate,sl_sz_bot,sl_sz_top,sl_vx0,sl_vy0,sl_vz0
0,282332,L,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,1.465449,1.646795,0.060431,-1.306652,-0.646412,0.073547,-0.115519,-1.143983,1.268112,0.545950
1,407845,R,-0.545069,-0.007018,0.675023,0.632901,-0.529787,0.626222,-0.13174,0.097922,...,-0.143343,-0.676061,0.603197,0.553478,-0.474859,-0.338700,-0.718141,0.849285,-0.532922,-1.366876
2,424144,L,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,-0.825003,1.696063,-1.122078,-1.501336,-0.378488,-0.258455,-0.586218,-1.454694,1.496090,1.513235
3,425772,R,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,425794,R,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
df.shape

(830, 218)

## Build Training Data
We're using the standardized pitch data. 
* All NA's are set to 0.
* Pitch handedness is now R=1, L=0

In [169]:
df_data_train = df.copy()
df_data_train['p_throws'] = df_data_train['p_throws'].map({'R': 1, 'L': 0})
df_data_train.fillna(0, inplace=True)
data_train = df_data_train.iloc[:,range(1, 218)].values.astype('float32')

## Create and train our model
[See the documentation of hyperparameters here](https://docs.aws.amazon.com/sagemaker/latest/dg/k-means-api-config.html)

In [170]:
from sagemaker import KMeans

num_clusters = 10
output_location = 's3://' + bucket + '/pitchtype'
role = get_execution_role()

kmeans = KMeans(role=role,
               train_instance_count=1,
               train_instance_type='ml.c4.xlarge',
               output_path=output_location,
               k=num_clusters)

In [171]:
%%time
job_name = 'pitch-test-kmeans-{}'.format(datetime.now().strftime("%Y%m%d%H%M%S"))
labels = np.empty(len(df_data_train.index))
result_set = kmeans.record_set(data_train)#, labels=labels)
kmeans.fit(result_set, job_name=job_name)
print('Here is the job name {}'.format(job_name))

2020-04-27 01:12:10 Starting - Starting the training job...
2020-04-27 01:12:11 Starting - Launching requested ML instances......
2020-04-27 01:13:21 Starting - Preparing the instances for training...
2020-04-27 01:14:10 Downloading - Downloading input data......
2020-04-27 01:15:10 Training - Training image download completed. Training in progress.
2020-04-27 01:15:10 Uploading - Uploading generated training model.Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/27/2020 01:15:07 INFO 140578399385408] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'_enable_profiler': u'false', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'local_lloyd_num_trials': u'auto', u'_log_level': u'info', u'_kvstore': u'auto', u'local_lloyd_init_method': u'kmeans++', u'force_dense': u'true', u'epochs': u'1', u'init_method': u'random', u'local_lloyd_tol': u'0.0001', u'local_lloyd


2020-04-27 01:15:16 Completed - Training job completed
Training seconds: 66
Billable seconds: 66
Here is the job name pitch-test-kmeans-20200427011209
CPU times: user 522 ms, sys: 24.7 ms, total: 547 ms
Wall time: 3min 42s


# Deploy the Model and Make Predictions
From https://www.bmc.com/blogs/amazon-sagemaker/

In [172]:
%%time
kmeans_predictor = kmeans.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-----------------!CPU times: user 298 ms, sys: 9.17 ms, total: 307 ms
Wall time: 8min 32s


In [187]:
result = kmeans_predictor.predict(data_train)
clusters = [r.label['closest_cluster'].float32_tensor.values[0] for r in result]
clusters[:10]

[2.0, 5.0, 2.0, 9.0, 0.0, 0.0, 5.0, 0.0, 1.0, 9.0]

# Output the Pitchers and their Cluster Membership

In [184]:
import s3fs

df_pitcher_membership = df[['pitcher']].copy()
df_pitcher_membership['cluster']=clusters

def _write_out_df(df, key):
    bytes_to_write = df.to_csv(None).encode()
    fs = s3fs.S3FileSystem()
    with fs.open('s3://{bucket}/{key}'.format(
            bucket=bucket_name,
            key=key
        ), 'wb') as f:
        f.write(bytes_to_write)
        
_write_out_df(df_pitcher_membership, f'pitchtype/{job_name}/pitcher_cluster_membership.csv')

# Viewing the results
Some more result viewing, like the details about the clusters

[See the documentation of deserilization here](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html#td-deserialization)

In [185]:
import os
model_key = f'{prefix}/{job_name}/output/model.tar.gz'

boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

2304

In [186]:
kmeans_model_params = mx.ndarray.load('model_algo-1')
cluster_centroids_kmeans = pd.DataFrame(kmeans_model_params[0].asnumpy())
cluster_centroids_kmeans.columns=df_data_train.columns[1:len(df_data_train.columns)]
cluster_centroids_kmeans

,p_throws,2f_ax,2f_ay,2f_az,2f_pct_usage,2f_pfx_x,2f_pfx_z,2f_plate_x,2f_plate_z,2f_release_extension,...,sl_release_extension,sl_release_pos_x,sl_release_pos_z,sl_release_speed,sl_release_spin_rate,sl_sz_bot,sl_sz_top,sl_vx0,sl_vy0,sl_vz0
0,9.644670e-01,-0.178254,-7.371738e-02,0.061187,0.085152,-0.185108,0.066459,-1.453121e-01,-0.053865,5.025877e-02,...,0.086050,-0.348510,0.136777,-6.207552e-02,0.005796,-0.044644,-0.003179,0.337102,0.057980,-0.036798
1,5.000001e-02,1.309089,-5.980115e-01,0.201254,0.142320,1.343369,0.372146,9.284890e-01,-0.037955,-5.080095e-01,...,-0.293915,0.442917,0.224854,-3.693717e-01,-0.006405,-0.079575,-0.106498,-0.398748,0.369415,0.131925
2,1.192093e-07,0.687806,-8.451711e-02,-0.097177,0.097085,0.686454,-0.073906,4.290763e-01,-0.017627,-3.832180e-03,...,-0.064690,1.226470,-0.044546,-1.534660e-01,-0.061840,0.005386,-0.091032,-1.212968,0.155808,0.074195
3,1.000000e+00,-0.033098,-7.288931e-02,-0.040243,0.069479,-0.038659,-0.020269,-5.879611e-03,0.088110,-2.897505e-02,...,-0.283708,-0.123385,-0.187217,-2.977845e-02,-0.097162,-0.112012,-0.124603,0.106271,0.030944,0.217015
4,1.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,...,-0.843966,-0.165261,0.009994,-1.540697e+00,-0.887619,0.399806,0.709734,0.064469,1.550804,0.859778
5,1.000000e+00,-0.374134,4.910500e-01,0.150886,0.140474,-0.349091,0.042992,-2.905415e-01,0.178951,-1.474119e-02,...,-0.024019,-0.422501,0.212902,6.322104e-01,0.089393,0.082233,0.034399,0.477446,-0.627175,-0.459397
6,1.000000e+00,-0.249759,-3.013914e+00,-0.769798,0.690561,-0.519815,0.416976,2.076755e-01,-0.158409,-1.289145e+00,...,0.000000,0.000000,0.000000,1.862645e-09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,1.000000e+00,-0.360175,-1.218212e-01,-0.428576,0.135838,-0.368228,-0.391597,-2.659051e-02,-0.040455,8.885404e-02,...,0.134926,-0.633305,-1.005179,-3.246852e-01,0.311132,-0.020933,-0.043656,0.516604,0.324208,0.818374
8,8.571429e-01,0.000000,3.725290e-09,0.000000,0.000000,0.000000,0.000000,-7.450581e-09,0.000000,-3.725290e-09,...,0.562721,-0.079125,-3.566267,-9.160632e-01,-0.328421,-0.318776,-0.385375,-0.084813,0.928679,2.549365
9,1.000000e+00,-0.018316,-1.360602e-01,-0.185542,0.003810,-0.031020,-0.160854,-1.354281e-01,0.146216,-2.722305e-01,...,-0.343919,-0.083105,0.186246,-6.017559e-01,-0.503449,0.193464,0.442727,0.074686,0.603449,0.090358
